In [9]:
# Install the packages\
%pip install requests openai python-dotenv praw

Note: you may need to restart the kernel to use updated packages.


In [18]:
from dotenv import load_dotenv
import os
import sys

sys.path.insert(0, '../code')

# Load the .env file
load_dotenv('../.env', override=True)

# Now you can access the variables using os.getenv
gpt4_azure_openai_endpoint = os.getenv("GPT4_AZURE_OPENAI_ENDPOINT")
gpt4_azure_openai_key = os.getenv("GPT4_AZURE_OPENAI_KEY")
gpt4_azure_openai_api_version = os.getenv("GPT4_AZURE_OPENAI_API_VERSION")
gpt4_deployment_name = os.getenv("GPT4_DEPLOYMENT_NAME")


gpt4


# Functions

In [11]:
# init tool array

tools = []
functions_map = {}

In [12]:
from reddit_functions import RedditClient

reddit = RedditClient()

fucntions = reddit.get_function_descriptions()

for f in fucntions:
    tools.append(f)

functions_map['subreddit_search'] = reddit.subreddit_search
functions_map['get_reddit_posts'] = reddit.get_reddit_posts

In [13]:
from file_fucntions import FileFunctions

file_functions = FileFunctions()
functions = file_functions.get_function_descriptions()
for f in functions:
    tools.append(f)

functions_map["write_to_file"] = file_functions.write_to_file
functions_map["read_file"] = file_functions.read_file

In [14]:
tools.append({"type": "code_interpreter"})

# Define Assistant

In [15]:
assistant_def = {
    'name' : "code_summarizer",
     'instructions' : """
    You are an AI assistant that can help with summarizing code. You can help with summarizing code mainframe code and write new java code.
    The summary should contain the following:
    - A brief description of the code
    - The purpose of the code
    - The input and output of the code
    - The expected behavior of the code
    - Any other relevant information
""",
'model': gpt4_deployment_name,
'azure_openai_key': gpt4_azure_openai_key,
'azure_openai_endpoint': gpt4_azure_openai_endpoint,
'azure_openai_api_version': gpt4_azure_openai_api_version
}


# Execution


In [16]:

from MiKoAssistant import MikoAssistant

verbose_output = True

mikoAssistant = MikoAssistant(assistant_def, tools, functions_map)

mikoAssistant.create_new_thread()


messages = mikoAssistant.send_message("Please create me a summary for the code in the 'demo.code' file.", verbose_output)

conversation = mikoAssistant.extract_message_content(messages, "output")
for c in conversation:
    print(c)

BadRequestError: Error code: 400 - {'error': {'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_found'}}